## 高斯过程
高斯过程（GP）是一种通用的监督学习方法，旨在解决回归和概率分类问题。

高斯过程的优点是：

 - 预测对观察值进行插值（至少对于常规内核而言）。

 - 该预测是概率性的（高斯），因此可以计算经验的置信区间，并根据这些经验来确定是否应该在某个感兴趣区域中对预测进行拟合（在线拟合，自适应拟合）。

 - 多功能：可以指定不同的内核。提供了通用内核，但是也可以指定自定义内核。

高斯过程的缺点包括：

 - 它们不是稀疏的，即，它们使用整个样本/特征信息来执行预测。

 - 它们在高维空间中会失去效率，也就是说，当特征数量超过几十个时
 
 
### 高斯过程回归（GPR）。

该实现基于Rasmussen和Williams提出的高斯机器学习过程算法（GPML）的算法2.1。除了标准的scikit-learn估算器API外GaussianProcessRegressor：允许进行预测而无需事先拟合（基于之前的GP）.提供了一个额外的方法sample_y（X），用于评估在给定输入下从GPR（先验或后验）提取的样本公开了一个方法log_marginal_likelihood（theta），该方法可在外部用于其他选择超参数的方式，例如通过马尔可夫链蒙特卡洛。

**参数:**
 - kernel:kernel instance, default=None;指定GP协方差函数的内核。如果未通过，则默认使用内核“ 1.0 * RBF（1.0）”。请注意，内核的超参数在拟合过程中已优化。
 - alpha:float or array-like of shape (n_samples), default=1e-10;拟合期间将值添加到核矩阵的对角线。较大的值对应于观测结果中增加的噪声级别。通过确保计算值形成正定矩阵，这也可以防止拟合过程中潜在的数值问题。如果传递了数组，则该数组必须具有与用于拟合的数据相同的条目数，并且用作与数据点有关的噪声级别。请注意，这等效于添加c = alpha的WhiteKernel。直接允许将噪声级别指定为参数主要是为了方便和与Ridge保持一致。
 - optimizer:“fmin_l_bfgs_b” or callable, default=”fmin_l_bfgs_b”;可以是内部支持的优化器之一，用于优化由字符串指定的内核参数，也可以是作为可调用函数传递的外部定义的优化器。如果传递了callable，则必须具有签名[具体详见](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor),默认情况下，使用scipy.optimize.minimize中的“ L-BGFS-B”算法。如果未传递任何参数，则内核参数保持固定。
 - n_restarts_optimizer:int, default=0;优化程序的重新启动次数，用于查找最大化对数边际可能性的内核参数。优化程序的第一次运行是从内核的初始参数执行的，其余的（如果有的话）是从theta采样到的log-uniform随机从允许的theta值空间进行的。如果大于0，则所有边界必须是有限的。请注意，n_restarts_optimizer == 0表示执行了一次运行。
 - normalize_y:boolean, optional (default: False):无论目标值y是否被归一化，目标值的均值和方差分别设置为等于0和1。对于使用零均值，单位方差先验的情况，建议使用此方法。请注意，在此实现中，在报告GP预测之前，将规范化反转。
 - copy_X_train:bool, default=True;如果为True，则训练数据的永久副本存储在对象中。否则，仅存储对训练数据的引用，如果对数据进行外部修改，则可能导致预测更改。
 - random_state:int or RandomState, default=None;随机数种子

In [1]:
from sklearn.datasets import make_friedman2
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import DotProduct, WhiteKernel
X, y = make_friedman2(n_samples=500, noise=0, random_state=0)
kernel = DotProduct() + WhiteKernel()
gpr = GaussianProcessRegressor(kernel=kernel,
        random_state=0).fit(X, y)
gpr.score(X, y)

0.3680293861017382

In [2]:
gpr.predict(X[:2,:], return_std=True)

(array([653.08792288, 592.16905327]), array([316.68016218, 316.65121679]))

### 高斯过程分类（GPC）

所述GaussianProcessClassifier器具高斯过程（GP）用于分类目的，更具体地用于概率分类，其中测试的预测采取类概率的形式。GaussianProcessClassifier将GP放在潜在函数$f$上，然后通过链接函数对其进行压缩以获得概率分类。潜在功能$f$是所谓的讨厌的功能，其值不被观察并且本身不相关。其目的是允许方便地制定模型，并且$f$ 在预测过程中被删除（集成）。GaussianProcessClassifier实现了逻辑链接函数，该函数无法解析地计算积分，但在二进制情况下很容易近似。

与回归设置相反，潜在函数的后验 $f$即使对于GP先验来说也不是高斯，因为高斯似然不适用于离散类标签。而是，使用与逻辑链接函数（logit）相对应的非高斯似然性。GaussianProcessClassifier基于Laplace逼近，以高斯近似非高斯后验。更多详细信息，请参见[RW2006]的第3章。

假设GP先验均值为零。通过传递内核对象来指定先验的协方差。内核的超参数在高斯过程回归器的拟合过程中通过基于传递的最大化对数边际可能性（LML）进行了优化optimizer。由于LML可能具有多个局部最优值，因此可以通过指定来反复启动优化器n_restarts_optimizer。始终从内核的初始超参数值开始进行第一次运行；随后的运行是从已从允许值范围内随机选择的超参数值进行的。如果初始超参数应保持固定，None则可以将其作为优化器传递。

GaussianProcessClassifier通过执行一对多休息或一对多一对一的训练和预测来支持多类分类。在相对于其余部分中，为每个类别安装一个二进制高斯过程分类器，并对其进行训练以将该类别与其余类别分开。在“ one_vs_one”中，为每对类装配一个二进制高斯过程分类器，训练该分类器以将这两个类分开。这些二进制预测变量的预测被组合为多类预测。有关更多详细信息，请参见多类分类部分。

在高斯过程分类的情况下，“ one_vs_one”在计算上可能更便宜，因为它必须解决仅涉及整个训练集的一个子集的许多问题，而不是解决整个数据集上较少的问题。由于高斯过程分类与数据集的大小成三次方缩放，因此这可能会更快。但是，请注意，“ one_vs_one”不支持预测概率估计，而仅支持简单预测。此外，**请注意，GaussianProcessClassifier尚未（内部）在内部实现真正的多类Laplace逼近，但是如上所述，该方法基于内部解决几个二进制分类任务的情况，这些任务使用“一对多”或“一对一”进行组合。**

基于拉普拉斯近似的高斯过程分类（GPC）。该实现基于Rasmussen和Williams提出的高斯机器学习过程（GPML）的算法3.1、3.2和5.1。在内部，拉普拉斯逼近用于通过高斯逼近非高斯后验。当前，该实现仅限于使用逻辑链接功能。对于多类别分类，安装了几个二进制的单反其余分类器。注意，该类因此未实现真正的多类拉普拉斯近似。

**参数**
 - kernel:kernel instance, default=None;指定GP协方差函数的内核。如果未通过，则默认使用内核“ 1.0 * RBF（1.0）”。请注意，内核的超参数在拟合过程中已优化。
 - optimizer:“fmin_l_bfgs_b” or callable, default=”fmin_l_bfgs_b”;可以是内部支持的优化器之一，用于优化由字符串指定的内核参数，也可以是作为可调用函数传递的外部定义的优化器。如果传递了callable，则必须具有签名[具体详见](https://scikit-learn.org/stable/modules/generated/sklearn.gaussian_process.GaussianProcessRegressor.html#sklearn.gaussian_process.GaussianProcessRegressor),默认情况下，使用scipy.optimize.minimize中的“ L-BGFS-B”算法。如果未传递任何参数，则内核参数保持固定。
 - n_restarts_optimizer:int, default=0;优化程序的重新启动次数，用于查找最大化对数边际可能性的内核参数。优化程序的第一次运行是从内核的初始参数执行的，其余的（如果有的话）是从theta采样到的log-uniform随机从允许的theta值空间进行的。如果大于0，则所有边界必须是有限的。请注意，n_restarts_optimizer == 0表示执行了一次运行。
 - max_iter_predict:int, default=100;预测过程中，牛顿方法中用于逼近后验的最大迭代次数。较小的值将减少计算时间，但结果会更差。
 - warm_start:bool, default=False;如果启用了热启动，则后验模式的拉普拉斯近似上的最后一次牛顿迭代的解将用作下一个_posterior_mode（）调用的初始化。在与超参数优化类似的问题上多次调用_posterior_mode时，这可以加快收敛速度。请参阅词汇表。   
 - copy_X_train:bool, default=True;如果为True，则训练数据的永久副本存储在对象中。否则，仅存储对训练数据的引用，如果对数据进行外部修改，则可能导致预测更改。
 - random_state:int or RandomState, default=None;随机数种子
 - multi_class:{‘one_vs_rest’, ‘one_vs_one’}, default=’one_vs_rest’;指定如何处理多类分类问题。支持的是“ one_vs_rest”和“ one_vs_one”。在“ one_vs_rest”中，为每个类别安装一个二进制高斯过程分类器，训练该分类器以将该类别与其余类别分开。在“ one_vs_one”中，为每对类装配一个二进制高斯过程分类器，训练该分类器以将这两个类分开。这些二进制预测变量的预测被组合为多类预测。注意，“ one_vs_one”不支持预测概率估计。
 - n_jobs:int, default=None;为邻居搜索运行的并行作业数。-1表示全部,但是可能会影响可视化编程的监控程序

In [3]:
from sklearn.datasets import load_iris
from sklearn.gaussian_process import GaussianProcessClassifier
from sklearn.gaussian_process.kernels import RBF
X, y = load_iris(return_X_y=True)
kernel = 1.0 * RBF(1.0)
gpc = GaussianProcessClassifier(kernel=kernel,
        random_state=0).fit(X, y)
gpc.score(X, y)

0.9866666666666667

In [4]:
gpc.predict_proba(X[:2,:])

array([[0.83548752, 0.03228706, 0.13222543],
       [0.79064206, 0.06525643, 0.14410151]])